# exp098
derivedデータを使ったbaselineの復元（紐づけ方法修正）+ multipath + 仰角マスク20度 + fixed 紐づけ再修正  
statusとの紐づけを近接に。statusと紐づかない場合はデータを使うように修正

In [1]:
# import library
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib_venn import venn2, venn2_circles
import seaborn as sns
from tqdm.notebook import tqdm
import pathlib
import plotly
import plotly.express as px
import itertools
import lightgbm as lgb
from optuna.integration import lightgbm as optuna_lgb
import simdkalman
import optuna
import pyproj
from pyproj import Proj, transform
from sklearn import metrics
from sklearn.metrics import roc_curve, precision_recall_curve, confusion_matrix, accuracy_score
pd.set_option('display.max_rows', 100)
from math import * 
import scipy.optimize as opt
import multiprocessing
pd.set_option('display.max_columns', 50)

In [2]:
import ipynb_path

def get_nb_name():
    nb_path = ipynb_path.get()
    nb_name = nb_path.rsplit('/',1)[1].replace('.ipynb','')
    return nb_name

In [3]:
# directory setting
nb_name = get_nb_name()
INPUT = '../input/google-smartphone-decimeter-challenge'
OUTPUT = '../output/' + nb_name
os.makedirs(OUTPUT, exist_ok=True)

# utils

In [4]:
def get_train_score(df, gt):
    gt = gt.rename(columns={'latDeg':'latDeg_gt', 'lngDeg':'lngDeg_gt'})
    df = df.merge(gt, on=['collectionName', 'phoneName', 'millisSinceGpsEpoch'], how='inner')
    # calc_distance_error
    df['err'] = calc_haversine(df['latDeg_gt'], df['lngDeg_gt'], df['latDeg'], df['lngDeg'])
    # calc_evaluate_score
    df['phone'] = df['collectionName'] + '_' + df['phoneName']
    res = df.groupby('phone')['err'].agg([percentile50, percentile95])
    res['p50_p90_mean'] = (res['percentile50'] + res['percentile95']) / 2 
    score = res['p50_p90_mean'].mean()
    return score

In [5]:
def calc_haversine(lat1, lon1, lat2, lon2):
    """Calculates the great circle distance between two points
    on the earth. Inputs are array-like and specified in decimal degrees.
    """
    RADIUS = 6_367_000
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2)**2 + \
        np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    dist = 2 * RADIUS * np.arcsin(a**0.5)
    return dist

In [6]:
def visualize_trafic(df, center, zoom=9):
    fig = px.scatter_mapbox(df,
                            
                            # Here, plotly gets, (x,y) coordinates
                            lat="latDeg",
                            lon="lngDeg",
                            
                            #Here, plotly detects color of series
                            color="phoneName",
                            labels="phoneName",
                            
                            zoom=zoom,
                            center=center,
                            height=600,
                            width=800)
    fig.update_layout(mapbox_style='stamen-terrain')
    fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})
    fig.update_layout(title_text="GPS trafic")
    fig.show()
    
def visualize_collection(df, collection):
    target_df = df[df['collectionName']==collection].copy()
    lat_center = target_df['latDeg'].mean()
    lng_center = target_df['lngDeg'].mean()
    center = {"lat":lat_center, "lon":lng_center}
    
    visualize_trafic(target_df, center)

In [7]:
# ground_truth
def get_ground_truth():
    p = pathlib.Path(INPUT)
    gt_files = list(p.glob('train/*/*/ground_truth.csv'))

    gts = []
    for gt_file in gt_files:
        gts.append(pd.read_csv(gt_file))
    ground_truth = pd.concat(gts)

    return ground_truth

In [8]:
def percentile50(x):
    return np.percentile(x, 50)
def percentile95(x):
    return np.percentile(x, 95)

In [9]:
class train_result:
    def __init__(self, df):
        self.df = df
        self.gt = get_ground_truth()
        self.bl = pd.read_csv(INPUT + '/' + 'baseline_locations_train.csv')
        
        self.gt = self.gt.rename(columns={'latDeg':'latDeg_gt', 'lngDeg':'lngDeg_gt'})
        self.df = self.df.merge(self.gt, on=['collectionName', 'phoneName', 'millisSinceGpsEpoch'], how='inner')
        self.df['phone'] = self.df['collectionName'] + '_' + self.df['phoneName']
        self.df['err'] =  calc_haversine(self.df['latDeg_gt'], self.df['lngDeg_gt'], self.df['latDeg'], self.df['lngDeg'])
        
        self.phone_res = self.calc_err('phone')
        self.clc_res = self.calc_err('collectionName')
        self.phonename_res = self.calc_err('phoneName')
        
    def calc_err(self, by):
        res = self.df.groupby(by)['err'].agg([percentile50, percentile95])
        res['p50_p90_mean'] = (res['percentile50'] + res['percentile95']) / 2
        return res
    
    @property
    def score(self):
        return self.phone_res['p50_p90_mean'].mean()
    @property
    def raw_data(self):
        return self.df
    @property
    def err(self):
        return self.phone_res
    @property
    def collection_err(self):
        return self.clc_res
    @property
    def phonename_err(self):
        return self.phonename_res
    
    def viz_map(self, collection, show_gt=True, show_bl=True):
        tmp = self.df[self.df['collectionName']==collection][['collectionName', 'phoneName', 'latDeg', 'lngDeg']]
        tmp2 = self.df[self.df['collectionName']==collection][['collectionName', 'phoneName', 'latDeg_gt', 'lngDeg_gt']]
        tmp2 = tmp2.rename(columns={'latDeg_gt':'latDeg', 'lngDeg_gt':'lngDeg'})
        tmp2['phoneName'] = tmp2['phoneName'] + '_GT'
        tmp3 = self.bl[self.bl['collectionName']==collection][['collectionName', 'phoneName', 'latDeg', 'lngDeg']]
        tmp3['phoneName'] = tmp3['phoneName'] + '_BL'
        
        if show_gt:
            tmp = tmp.append(tmp2)
        if show_bl:
            tmp = tmp.append(tmp3)
        visualize_collection(tmp, collection)

In [10]:
def get_data():
    base_train = pd.read_csv(INPUT + '/' + 'baseline_locations_train.csv')
    base_test = pd.read_csv(INPUT + '/' + 'baseline_locations_test.csv')
    sample_sub = pd.read_csv(INPUT + '/' + 'sample_submission.csv')
    ground_truth = pd.read_csv(INPUT + '/prep/ground_truth_train.csv')
    return base_train, base_test, sample_sub, ground_truth

In [11]:
def ecef2lla(x, y, z):
    # x, y and z are scalars or vectors in meters
    x = np.array([x]).reshape(np.array([x]).shape[-1], 1)
    y = np.array([y]).reshape(np.array([y]).shape[-1], 1)
    z = np.array([z]).reshape(np.array([z]).shape[-1], 1)

    a=6378137
    a_sq=a**2
    e = 8.181919084261345e-2
    e_sq = 6.69437999014e-3

    f = 1/298.257223563
    b = a*(1-f)

    # calculations:
    r = np.sqrt(x**2 + y**2)
    ep_sq  = (a**2-b**2)/b**2
    ee = (a**2-b**2)
    f = (54*b**2)*(z**2)
    g = r**2 + (1 - e_sq)*(z**2) - e_sq*ee*2
    c = (e_sq**2)*f*r**2/(g**3)
    s = (1 + c + np.sqrt(c**2 + 2*c))**(1/3.)
    p = f/(3.*(g**2)*(s + (1./s) + 1)**2)
    q = np.sqrt(1 + 2*p*e_sq**2)
    r_0 = -(p*e_sq*r)/(1+q) + np.sqrt(0.5*(a**2)*(1+(1./q)) - p*(z**2)*(1-e_sq)/(q*(1+q)) - 0.5*p*(r**2))
    u = np.sqrt((r - e_sq*r_0)**2 + z**2)
    v = np.sqrt((r - e_sq*r_0)**2 + (1 - e_sq)*z**2)
    z_0 = (b**2)*z/(a*v)
    h = u*(1 - b**2/(a*v))
    phi = np.arctan((z + ep_sq*z_0)/r)
    lambd = np.arctan2(y, x)

    return phi*180/np.pi, lambd*180/np.pi, h

# baselineの再作成

In [12]:
def prepare_calc_baseline(df):
    light_speed = 299_792_458
    omega_e = 7.2921151467e-5
    
    # Corrected pseudorange according to data instructions
    df['correctedPrM'] = df['rawPrM'] + \
                         df['satClkBiasM'] - \
                         df['isrbM'] - \
                         df['ionoDelayM'] - \
                         df['tropoDelayM']
    
    # Time it took for signal to travel
    df['transmissionTimeSeconds'] = df['correctedPrM'] / light_speed
    
    # Compute true sat positions at arrival time
    df['xSatPosMRotated'] = \
        np.cos(omega_e * df['transmissionTimeSeconds']) * df['xSatPosM'] \
        + np.sin(omega_e * df['transmissionTimeSeconds']) * df['ySatPosM']

    df['ySatPosMRotated'] = \
        - np.sin(omega_e * df['transmissionTimeSeconds']) * df['xSatPosM'] \
        + np.cos(omega_e * df['transmissionTimeSeconds']) * df['ySatPosM']

    df['zSatPosMRotated'] = df['zSatPosM']
    
    # Uncertainty weight for the WLS method
    df['uncertaintyWeight'] = 1 / df['rawPrUncM']
    
    return df

In [13]:
def calc_baseline_point(df):

    def distance(sat_pos, x):
        sat_pos_diff = sat_pos.copy(deep=True)

        sat_pos_diff['xSatPosMRotated'] = sat_pos_diff['xSatPosMRotated'] - x[0]
        sat_pos_diff['ySatPosMRotated'] = sat_pos_diff['ySatPosMRotated'] - x[1]
        sat_pos_diff['zSatPosMRotated'] = sat_pos_diff['zSatPosMRotated'] - x[2]

        sat_pos_diff['d'] = sat_pos_diff['uncertaintyWeight'] * \
                            (np.sqrt((sat_pos_diff['xSatPosMRotated']**2 + sat_pos_diff['ySatPosMRotated']**2 + sat_pos_diff['zSatPosMRotated']**2)) + \
                             x[3] - sat_pos_diff['correctedPrM'])

        return sat_pos_diff['d']

    def distance_fixed_satpos(x):
        return distance(df[['xSatPosMRotated', 'ySatPosMRotated', 'zSatPosMRotated', 'correctedPrM', 'uncertaintyWeight']], x)
    
    x0 = [0,0,0,0]
    opt_res = opt.least_squares(distance_fixed_satpos, x0)
    # Optimiser yields a position in the ECEF coordinates
    opt_res_pos = opt_res.x
    
    # ECEF position to lat/long
    wls_estimated_pos = ecef2lla(*opt_res_pos[:3])
    wls_estimated_pos = np.squeeze(wls_estimated_pos)
    
    return wls_estimated_pos[0], wls_estimated_pos[1]

In [14]:
def get_derived_data(train_test, collection, phonename):
    derived = pd.read_csv(INPUT + f'/{train_test}/{collection}/{phonename}/{phonename}_derived.csv')
    raw = pd.read_csv(INPUT + f'/prep/gnss/{train_test}/{collection}/{phonename}/Raw.csv')
    status = pd.read_csv(INPUT + f'/prep/gnss/{train_test}/{collection}/{phonename}/Status.csv')
    
    # Assume we've loaded a dataframe from _GnssLog.txt for only lines beginning with "Raw", we denote this df_raw. Next, assume we've loaded a dataframe from _derived.csv. We denote this df_derived.

    # Create a new column in df_raw that corresponds to df_derived['MillisSinceGpsEpoch']
    raw['millisSinceGpsEpoch'] = np.floor( (raw['TimeNanos'] - raw['FullBiasNanos']) / 1000000.0).astype(int)
    
    # Change each value in df_derived['MillisSinceGpsEpoch'] to be the prior epoch.
    raw_timestamps = raw['millisSinceGpsEpoch'].unique()
    derived_timestamps = derived['millisSinceGpsEpoch'].unique()

    # The timestamps in derived are one epoch ahead. We need to map each epoch
    # in derived to the prior one (in Raw).
    indexes = np.searchsorted(raw_timestamps, derived_timestamps)
    from_t_to_fix_derived = dict(zip(derived_timestamps, raw_timestamps[indexes-1]))
    derived['millisSinceGpsEpoch'] = np.array(list(map(lambda v: from_t_to_fix_derived[v], derived['millisSinceGpsEpoch'])))

    # Compute signal_type in df_raw.
    # Map from constellation id to frequencies and signals.
    CONSTEL_FREQ_TABLE = {
        0: {'UNKNOWN': (0, 999999999999)},
        1: {
            'GPS_L1': (1563000000, 1587000000),
            'GPS_L2': (1215000000, 1240000000),
            'GPS_L5': (1164000000, 1189000000)
        },
        3: {
            'GLO_G1': (1593000000, 1610000000),
            'GLO_G2': (1237000000, 1254000000)
        },
        4: {
            'QZS_J1': (1563000000, 1587000000),
            'QZS_J2': (1215000000, 1240000000),
            'QZS_J5': (1164000000, 1189000000)
        },
        5: {
            'BDS_B1C': (1569000000, 1583000000),
            'BDS_B1I': (1553000000, 1568990000),
            'BDS_B2A': (1164000000, 1189000000),
            'BDS_B2B': (1189000000, 1225000000)
        },
        6: {
            'GAL_E1': (1559000000, 1591000000),
            'GAL_E5A': (1164000000, 1189000000),
            'GAL_E5B': (1189000000, 1218000000),
            'GAL_E6': (1258000000, 1300000000)
        },
        7: {
            'IRN_S': (2472000000, 2512000000),
            'IRN_L5': (1164000000, 1189000000)
        },
    }

    def SignalTypeFromConstellationAndFequency(constel, freq_hz):
        'Returns the signal type as a string for the given constellation and frequency.'
        freqs = CONSTEL_FREQ_TABLE.get(constel, {})
        for id_freq_range in freqs.items():
            rng = id_freq_range[1]
            if rng[0] <= freq_hz <= rng[1]:
                return id_freq_range[0]
        return 'UNKNOWN'

    signal_types = itertools.chain(*[c.keys() for c in CONSTEL_FREQ_TABLE.values()])
    sig_type_cat = pd.api.types.CategoricalDtype(categories=signal_types)
    raw['signalType'] = raw.apply(lambda r: SignalTypeFromConstellationAndFequency(r.ConstellationType, r.CarrierFrequencyHz), axis=1).astype(sig_type_cat)

    # Fix QZS Svids issue. 

    # The SVID of any QZS sat in derived may be changed. Since it may be a many to one relationship, we'll need to adjust the values in Raw.
    new_to_old = {1:(183, 193), 2:(184, 194, 196), 3:(187, 189, 197, 199), 4:(185, 195, 200)}
    # Maps original svid to new svid for only ConstellationType=4.
    old_to_new={}
    for new_svid, old_svids in new_to_old.items():
        for s in old_svids:
            old_to_new[s] = new_svid
    raw['svid'] = raw.apply(lambda r: old_to_new.get(r.Svid, r.Svid) if r.ConstellationType == 4 else r.Svid, axis=1)
    del raw['collectionName']
    del raw['phoneName']
    derived = derived.merge(raw, on=['millisSinceGpsEpoch', 'svid', 'signalType'], how='left')
    
    # status
    status['millisSinceGpsEpoch'] = status['UnixTimeMillis'] - 315964800000 + 18000
    status['svid'] = status.apply(lambda r: old_to_new.get(r.Svid, r.Svid) if r.ConstellationType == 4 else r.Svid, axis=1)
    status['signalType'] = status.apply(lambda r: SignalTypeFromConstellationAndFequency(r.ConstellationType, r.CarrierFrequencyHz), axis=1).astype(sig_type_cat)
    del status['collectionName']
    del status['phoneName']
    status = status.drop_duplicates(subset=['svid', 'signalType', 'millisSinceGpsEpoch'])
    
    sv_sig = derived[['svid', 'signalType']].drop_duplicates()
    output_df = pd.DataFrame()
    for svid, signal in zip(sv_sig['svid'], sv_sig['signalType']):
        derived_tmp = derived[(derived['svid']==svid)&(derived['signalType']==signal)].copy()
        status_tmp = status[(status['svid']==svid)&(status['signalType']==signal)].copy()
        del status_tmp['svid']
        del status_tmp['signalType']
        output_tmp = pd.merge_asof(derived_tmp, status_tmp, on='millisSinceGpsEpoch', direction='nearest')
        output_df = output_df.append(output_tmp)
    
    return output_df

In [15]:
def calc_baseline(args):
    s_th = 10
    
    phone, df = args
    collection = phone.split('_')[0]
    phonename = phone.split('_')[1]
    derived = get_derived_data('train', collection, phonename)
    derived = prepare_calc_baseline(derived)
    
    derived = derived[~(derived['MultipathIndicator']==1)].copy()
    derived = derived[~(derived['ElevationDegrees']<=20)].copy()
    derived = derived[~(derived['UsedInFix']==0)].copy()
    
    idx = list(df.index)
    s_list = []
    n_list = []
    lat_list = []
    lng_list = []
    unc_mean_list = []
    unc_max_list = []
    
    for j,i in enumerate(idx):
        s = df.at[i, 'millisSinceGpsEpoch']
        tmp = derived[(derived['millisSinceGpsEpoch']>=s-s_th)&(derived['millisSinceGpsEpoch']<=s+s_th)].copy()
        n = tmp['svid'].nunique()
        s_list.append(s)
        n_list.append(n)
        
        if n < 4:    
            lat_list.append(np.nan)
            lng_list.append(np.nan)
            unc_mean_list.append(np.nan)
            unc_max_list.append(np.nan)        
        
        else:
            res = calc_baseline_point(tmp)
            lat_list.append(res[0])
            lng_list.append(res[1])
            unc_mean_list.append(tmp['uncertaintyWeight'].mean())
            unc_max_list.append(tmp['uncertaintyWeight'].max())
    
    output_df = pd.DataFrame()
    output_df['millisSinceGpsEpoch'] = s_list
    output_df['latDeg'] = lat_list
    output_df['lngDeg'] = lng_list
    output_df['n'] = n_list
    output_df['unc_mean'] = unc_mean_list
    output_df['unc_max'] = unc_max_list
    output_df['collectionName'] = collection
    output_df['phoneName'] = phonename
    output_df['phone'] = phone
    
    return output_df

In [16]:
def calc_baseline_test(args):
    s_th = 10
    
    phone, df = args
    collection = phone.split('_')[0]
    phonename = phone.split('_')[1]
    derived = get_derived_data('test', collection, phonename)
    derived = prepare_calc_baseline(derived)
    
    derived = derived[~(derived['MultipathIndicator']==1)].copy()
    derived = derived[~(derived['ElevationDegrees']<=20)].copy()
    derived = derived[~(derived['UsedInFix']==0)].copy()
    
    idx = list(df.index)
    s_list = []
    n_list = []
    lat_list = []
    lng_list = []
    unc_mean_list = []
    unc_max_list = []
    
    for j,i in enumerate(idx):
        s = df.at[i, 'millisSinceGpsEpoch']
        tmp = derived[(derived['millisSinceGpsEpoch']>=s-s_th)&(derived['millisSinceGpsEpoch']<=s+s_th)].copy()
        n = tmp['svid'].nunique()
        s_list.append(s)
        n_list.append(n)
        
        if n < 4:    
            lat_list.append(np.nan)
            lng_list.append(np.nan)
            unc_mean_list.append(np.nan)
            unc_max_list.append(np.nan)        
        
        else:
            res = calc_baseline_point(tmp)
            lat_list.append(res[0])
            lng_list.append(res[1])
            unc_mean_list.append(tmp['uncertaintyWeight'].mean())
            unc_max_list.append(tmp['uncertaintyWeight'].max())
    
    output_df = pd.DataFrame()
    output_df['millisSinceGpsEpoch'] = s_list
    output_df['latDeg'] = lat_list
    output_df['lngDeg'] = lng_list
    output_df['n'] = n_list
    output_df['unc_mean'] = unc_mean_list
    output_df['unc_max'] = unc_max_list
    output_df['collectionName'] = collection
    output_df['phoneName'] = phonename
    output_df['phone'] = phone
    
    return output_df

In [17]:
df = get_derived_data('train', '2021-04-28-US-MTV-1', 'Pixel5')

In [18]:
df['UsedInFix'].value_counts(dropna=False)

1.0    29677
0.0    19414
NaN       10
Name: UsedInFix, dtype: int64

In [19]:
df = df[~(df['MultipathIndicator']==1)].copy()

In [20]:
df = df[~(df['ElevationDegrees']<=20)].copy()

In [21]:
df = df[~(df['UsedInFix']==0)].copy()

In [22]:
df

collectionName phoneName  millisSinceGpsEpoch  constellationType  \
11  2021-04-28-US-MTV-1    Pixel5        1303683574429                  6   
12  2021-04-28-US-MTV-1    Pixel5        1303683575429                  6   
13  2021-04-28-US-MTV-1    Pixel5        1303683576429                  6   
14  2021-04-28-US-MTV-1    Pixel5        1303683577429                  6   
15  2021-04-28-US-MTV-1    Pixel5        1303683578429                  6   
..                  ...       ...                  ...                ...   
5   2021-04-28-US-MTV-1    Pixel5        1303685542429                  3   
6   2021-04-28-US-MTV-1    Pixel5        1303685543429                  3   
7   2021-04-28-US-MTV-1    Pixel5        1303685544429                  3   
8   2021-04-28-US-MTV-1    Pixel5        1303685545429                  3   
9   2021-04-28-US-MTV-1    Pixel5        1303685545429                  3   

    svid signalType  receivedSvTimeInGpsNanos     xSatPosM      ySatPosM  \
11    13     GAL_E1       1303683574352084247 -7302972.174 -1.813378e+07   
12    13     GAL_E1       1303683575352083645 -7300906.042 -1.813293e+07   
13    13     GAL_E1       1303683576352083106 -7298839.712 -1.813208e+07   
14    13     GAL_E1       1303683577352082543 -7296773.185 -1.813123e+07   
15    13     GAL_E1       1303683578352081982 -7294706.461 -1.813038e+07   
..   ...        ...                       ...          ...           ...   
5     12     GLO_G1       1303685542351374224 -6934603.893  9.136897e+06   
6     12     GLO_G1       1303685543351376751 -6936326.184  9.134300e+06   
7     12     GLO_G1       1303685544351379478 -6938048.727  9.131703e+06   
8     12     GLO_G1       1303685545351382048 -6939771.517  9.129106e+06   
9     12     GLO_G1       1303685546351384721 -6941494.558  9.126509e+06   

        zSatPosM  xSatVelMps  ySatVelMps  zSatVelMps  satClkBiasM  \
11  2.222515e+07    2066.035     849.250    1372.986   124615.524   
12  2.222652e+07    2066.232     849.130    1372.644   124615.524   
13  2.222790e+07    2066.429     849.011    1372.303   124615.524   
14  2.222927e+07    2066.626     848.892    1371.961   124615.524   
15  2.223064e+07    2066.823     848.773    1371.619   124615.524   
..           ...         ...         ...         ...          ...   
5   2.280470e+07   -1722.163   -2597.136     519.213    67339.597   
6   2.280522e+07   -1722.413   -2597.055     518.667    67339.598   
7   2.280574e+07   -1722.662   -2596.974     518.121    67339.599   
8   2.280626e+07   -1722.911   -2596.893     517.574    67339.600   
9   2.280678e+07   -1723.161   -2596.811     517.028    67339.601   

    satClkDriftMps        rawPrM  rawPrUncM     isrbM  ionoDelayM  \
11           0.000  2.333164e+07      9.294  -441.443       3.839   
12           0.000  2.333182e+07      8.094  -441.268       3.839   
13           0.000  2.333198e+07      6.296  -447.294       3.839   
14           0.000  2.333215e+07      7.195  -440.485       3.839   
15           0.000  2.333232e+07      7.195  -443.139       3.840   
..             ...           ...        ...       ...         ...   
5            0.001  2.354450e+07     20.985  1126.562       8.250   
6            0.001  2.354374e+07     11.992  1141.824       8.249   
7            0.001  2.354292e+07     13.790  1140.580       8.248   
8            0.001  2.354215e+07     13.790  1139.678       8.247   
9            0.001  2.354135e+07     18.287  1137.044       8.247   

    tropoDelayM  utcTimeMillis         TimeNanos  LeapSecond  \
11        2.596  1619648356429  4259733502000000          18   
12        2.596  1619648357429  4259734502000000          18   
13        2.596  1619648358429  4259735502000000          18   
14        2.596  1619648359429  4259736502000000          18   
15        2.596  1619648360429  4259737502000000          18   
..          ...            ...               ...         ...   
5        13.307  1619650324429  4261701502000000          18   
6

In [23]:
train, test, sub, gt = get_data()

In [24]:
target = ['2021-04-22-US-SJC-1_Pixel4', '2021-04-22-US-SJC-1_SamsungS20Ultra',
          '2021-04-28-US-SJC-1_Pixel4', '2021-04-28-US-SJC-1_SamsungS20Ultra',
          '2021-04-29-US-SJC-2_Pixel4', '2021-04-29-US-SJC-2_SamsungS20Ultra',
          '2021-04-28-US-MTV-1_Pixel4', '2021-04-28-US-MTV-1_Pixel5', '2021-04-28-US-MTV-1_SamsungS20Ultra',
          '2021-04-29-US-MTV-1_Pixel4', '2021-04-29-US-MTV-1_Pixel5', '2021-04-29-US-MTV-1_SamsungS20Ultra']
train = train[train['phone'].isin(target)].copy()

In [25]:
processes = multiprocessing.cpu_count()
with multiprocessing.Pool(processes=processes) as pool:
    dfs = pool.imap_unordered(calc_baseline, train.groupby('phone'))
    dfs = tqdm(dfs)
    dfs = list(dfs)
result = pd.concat(dfs)

0it [00:00, ?it/s]

In [26]:
train_tmp = train[['phone', 'millisSinceGpsEpoch', 'latDeg', 'lngDeg']].copy()
train_tmp.columns = ['phone', 'millisSinceGpsEpoch', 'latDeg_bl', 'lngDeg_bl']
result = result.merge(train_tmp, on=['phone', 'millisSinceGpsEpoch'], how='left')

In [27]:
gt['phone'] = gt['collectionName'] + '_' + gt['phoneName']
gt_tmp = gt[['phone', 'millisSinceGpsEpoch', 'latDeg', 'lngDeg']].copy()
gt_tmp.columns = ['phone', 'millisSinceGpsEpoch', 'latDeg_gt', 'lngDeg_gt']
result = result.merge(gt_tmp, on=['phone', 'millisSinceGpsEpoch'], how='left')

In [28]:
result['rb_bl_err'] = calc_haversine(result['latDeg_bl'], result['lngDeg_bl'], result['latDeg'], result['lngDeg'])
result['rb_gt_err'] = calc_haversine(result['latDeg_gt'], result['lngDeg_gt'], result['latDeg'], result['lngDeg'])
result['bl_gt_err'] = calc_haversine(result['latDeg_gt'], result['lngDeg_gt'], result['latDeg_bl'], result['lngDeg_bl'])

In [29]:
for phone in result['phone'].unique():
    tmp = result[result['phone']==phone].copy()
    null_rate = tmp['latDeg'].isnull().sum() / len(tmp)
    print(phone, null_rate)

2021-04-29-US-MTV-1_Pixel5 0.0006265664160401002
2021-04-29-US-MTV-1_Pixel4 0.0006242197253433209
2021-04-29-US-MTV-1_SamsungS20Ultra 0.0006027727546714888
2021-04-28-US-MTV-1_Pixel5 0.0005037783375314861
2021-04-28-US-SJC-1_Pixel4 0.0019860973187686196
2021-04-28-US-MTV-1_SamsungS20Ultra 0.0005047955577990914
2021-04-28-US-SJC-1_SamsungS20Ultra 0.0009601536245799327
2021-04-28-US-MTV-1_Pixel4 0.0005002501250625312
2021-04-29-US-SJC-2_Pixel4 0.0017167381974248926
2021-04-29-US-SJC-2_SamsungS20Ultra 0.0008438818565400844
2021-04-22-US-SJC-1_SamsungS20Ultra 0.0014154281670205238
2021-04-22-US-SJC-1_Pixel4 0.001384083044982699


In [30]:
result_grouped = result.dropna().groupby('phone')[['rb_bl_err', 'rb_gt_err', 'bl_gt_err']].agg([percentile50, percentile95]).reset_index()
result_grouped.columns = ['phone', 'rb_bl_err_p50', 'rb_bl_err_p95', 'rb_gt_err_p50', 'rb_gt_err_p95', 'bl_gt_err_p50', 'bl_gt_err_p95']
result_grouped['rb_score'] = (result_grouped['rb_gt_err_p95'] + result_grouped['rb_gt_err_p50']) / 2
result_grouped['bl_score'] = (result_grouped['bl_gt_err_p95'] + result_grouped['bl_gt_err_p50']) / 2
result_grouped

phone  rb_bl_err_p50  rb_bl_err_p95  \
0            2021-04-22-US-SJC-1_Pixel4       1.654703      20.932590   
1   2021-04-22-US-SJC-1_SamsungS20Ultra       1.371024      14.856251   
2            2021-04-28-US-MTV-1_Pixel4       1.075283       4.916206   
3            2021-04-28-US-MTV-1_Pixel5       0.739491       2.875719   
4   2021-04-28-US-MTV-1_SamsungS20Ultra       0.778870       3.565681   
5            2021-04-28-US-SJC-1_Pixel4       2.213044      14.028654   
6   2021-04-28-US-SJC-1_SamsungS20Ultra       1.709623      23.174068   
7            2021-04-29-US-MTV-1_Pixel4       2.971406      13.828735   
8            2021-04-29-US-MTV-1_Pixel5       4.361243      16.241658   
9   2021-04-29-US-MTV-1_SamsungS20Ultra       0.809860       5.696548   
10           2021-04-29-US-SJC-2_Pixel4       3.023018      24.584521   
11  2021-04-29-US-SJC-2_SamsungS20Ultra       1.753777      15.797926   

    rb_gt_err_p50  rb_gt_err_p95  bl_gt_err_p50  bl_gt_err_p95   rb_score  \
0        7.091520      41.889788       6.997472      42.133985  24.490654   
1        7.356437      28.519701       7.393606      28.045083  17.938069   
2        2.046631       7.673834       2.164321       7.365087   4.860233   
3        1.615985       4.587182       1.650982       3.956857   3.101583   
4        3.539691      11.542554       3.397275      11.083685   7.541123   
5        4.796491      24.919666       4.167358      24.437963  14.858078   
6        5.911945      34.164893       5.727631      34.629760  20.038419   
7        6.833109      19.786113       6.026172      16.005119  13.309611   
8        7.758773      20.343813       6.595427      15.593213  14.051293   
9        6.475850      12.990243       6.344816      13.739767   9.733046   
10       5.430611      31.349096       4.817066      35.498057  18.389854   
11       6.277723      28.350295       5.944472      32.948014  17.314009   

     bl_score  
0   24.565728  
1   17.719344  
2    4.764704  
3    2.803919  
4    7.240480  
5   14.302661  
6   20.178696  
7   11.015645  
8   11.094320  
9   10.042291  
10  20.157562  
11  19.446243

In [31]:
result.to_csv(OUTPUT + '/result.csv', index=False)
result_grouped.to_csv(OUTPUT + '/result_grouped.csv', index=False)

In [32]:
result_grouped[['rb_score', 'bl_score']].mean()

rb_score    13.802164
bl_score    13.610966
dtype: float64

In [33]:
target = ['2021-04-22-US-SJC-1_Pixel4', '2021-04-22-US-SJC-1_SamsungS20Ultra',
          '2021-04-28-US-SJC-1_Pixel4', '2021-04-28-US-SJC-1_SamsungS20Ultra',
          '2021-04-29-US-SJC-2_Pixel4', '2021-04-29-US-SJC-2_SamsungS20Ultra']
result_grouped[result_grouped['phone'].isin(target)][['rb_score', 'bl_score']].mean()

rb_score    18.838180
bl_score    19.395039
dtype: float64

In [34]:
target = ['2021-04-28-US-MTV-1_Pixel4', '2021-04-28-US-MTV-1_Pixel5', '2021-04-28-US-MTV-1_SamsungS20Ultra',
          '2021-04-29-US-MTV-1_Pixel4', '2021-04-29-US-MTV-1_Pixel5', '2021-04-29-US-MTV-1_SamsungS20Ultra']
result_grouped[result_grouped['phone'].isin(target)][['rb_score', 'bl_score']].mean()

rb_score    8.766148
bl_score    7.826893
dtype: float64

In [35]:
result

millisSinceGpsEpoch     latDeg      lngDeg   n  unc_mean   unc_max  \
0            1303768902678  37.395835 -122.103000   7  0.242380  0.476417   
1            1303768903670  37.395754 -122.102841   6  0.292165  0.555864   
2            1303768904681  37.395859 -122.102940   7  0.275391  0.555864   
3            1303768905673  37.395834 -122.103073   7  0.292081  0.555864   
4            1303768906686  37.395842 -122.102998   7  0.288067  0.555864   
...                    ...        ...         ...  ..       ...       ...   
25330        1303163462446  37.334590 -121.899390  10  0.403392  0.834028   
25331        1303163463446  37.334587 -121.899387  10  0.403058  0.667111   
25332        1303163464446  37.334599 -121.899423  11  0.377574  0.667111   
25333        1303163465446  37.334622 -121.899391  11  0.412523  0.667111   
25334        1303163466446        NaN         NaN   0       NaN       NaN   

            collectionName phoneName                       phone  latDeg_bl  \
0      2021-04-29-US-MTV-1    Pixel5  2021-04-29-US-MTV-1_Pixel5  37.395843   
1      2021-04-29-US-MTV-1    Pixel5  2021-04-29-US-MTV-1_Pixel5  37.395757   
2      2021-04-29-US-MTV-1    Pixel5  2021-04-29-US-MTV-1_Pixel5  37.395866   
3      2021-04-29-US-MTV-1    Pixel5  2021-04-29-US-MTV-1_Pixel5  37.395827   
4      2021-04-29-US-MTV-1    Pixel5  2021-04-29-US-MTV-1_Pixel5  37.395780   
...                    ...       ...                         ...        ...   
25330  2021-04-22-US-SJC-1    Pixel4  2021-04-22-US-SJC-1_Pixel4  37.334590   
25331  2021-04-22-US-SJC-1    Pixel4  2021-04-22-US-SJC-1_Pixel4  37.334579   
25332  2021-04-22-US-SJC-1    Pixel4  2021-04-22-US-SJC-1_Pixel4  37.334589   
25333  2021-04-22-US-SJC-1    Pixel4  2021-04-22-US-SJC-1_Pixel4  37.334621   
25334  2021-04-22-US-SJC-1    Pixel4  2021-04-22-US-SJC-1_Pixel4  37.334608   

        lngDeg_bl  latDeg_gt   lngDeg_gt  rb_bl_err  rb_gt_err  bl_gt_err  
0     -122.103038  37.395825 -122.102925   3.461438   6.684964  10.140319  
1     -122.102828  37.395825 -122.102925   1.137618  10.867007  11.430197  
2     -122.102987  37.395825 -122.102925   4.205162   3.978334   7.092634  
3     -122.102937  37.395825 -122.102925  12.075379  13.095244   1.027007  
4     -122.102861  37.395825 -122.102925  13.909743   6.682114   7.573183  
...           ...        ...         ...        ...        ...        ...  
25330 -121.899411  37.334599 -121.899427   1.852724   3.464328   1.747994  
25331 -121.899403  37.334599 -121.899427   1.676169   3.785345   3.040781  
25332 -121.899423  37.334599 -121.899427   1.123740   0.387692   1.168164  
25333 -121.899401  37.334599 -121.899427   0.864847   4.067802   3.407626  
25334 -121.899407  37.334598 -121.899427        NaN        NaN   2.045893  

[25335 rows x 16 columns]

# test

In [36]:
target = ['2021-04-22-US-SJC-2_SamsungS20Ultra', '2021-04-29-US-SJC-3_Pixel4', '2021-04-29-US-SJC-3_SamsungS20Ultra']
test = test[test['phone'].isin(target)].copy()

In [37]:
processes = multiprocessing.cpu_count()
with multiprocessing.Pool(processes=processes) as pool:
    dfs = pool.imap_unordered(calc_baseline_test, test.groupby('phone'))
    dfs = tqdm(dfs)
    dfs = list(dfs)
result = pd.concat(dfs)

0it [00:00, ?it/s]

In [38]:
result.to_csv(OUTPUT + '/result_test.csv', index=False)

In [39]:
for phone in result['phone'].unique():
    tmp = result[result['phone']==phone].copy()
    null_rate = tmp['latDeg'].isnull().sum() / len(tmp)
    print(phone, null_rate)

2021-04-29-US-SJC-3_Pixel4 0.00202122283981809
2021-04-29-US-SJC-3_SamsungS20Ultra 0.00048520135856380397
2021-04-22-US-SJC-2_SamsungS20Ultra 0.0012908777969018934
